In [1]:
import cassandra

In [2]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['cassandra'])
    session = cluster.connect()
    print("Connection Established")
except Exception as e:
    print("Connection Failed")

Connection Established


In [14]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkify
WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}"""
)

In [3]:
import os
import pandas as pd
import csv
import glob

In [5]:
session.execute("""DESCRIBE KEYSPACES""")

In [10]:
session.set_keyspace('sparkify')

In [30]:
session.execute("""
    CREATE TABLE IF NOT EXISTS session_songs
    (sessionId int, itemInSession int, artist text, song_title text, song_length float,
    PRIMARY KEY(sessionId, itemInSession))
    """)

In [10]:
print(os.getcwd())

/home/jovyan


In [4]:
path = os.getcwd()+'/event_data'
file_path_list = glob.glob(path+'/*')

In [5]:
print(file_path_list)

['/home/jovyan/event_data/2018-11-20-events.csv', '/home/jovyan/event_data/2018-11-28-events.csv', '/home/jovyan/event_data/2018-11-05-events.csv', '/home/jovyan/event_data/2018-11-25-events.csv', '/home/jovyan/event_data/2018-11-16-events.csv', '/home/jovyan/event_data/2018-11-08-events.csv', '/home/jovyan/event_data/2018-11-24-events.csv', '/home/jovyan/event_data/2018-11-19-events.csv', '/home/jovyan/event_data/2018-11-29-events.csv', '/home/jovyan/event_data/2018-11-01-events.csv', '/home/jovyan/event_data/2018-11-09-events.csv', '/home/jovyan/event_data/2018-11-30-events.csv', '/home/jovyan/event_data/2018-11-10-events.csv', '/home/jovyan/event_data/2018-11-17-events.csv', '/home/jovyan/event_data/2018-11-15-events.csv', '/home/jovyan/event_data/2018-11-12-events.csv', '/home/jovyan/event_data/2018-11-22-events.csv', '/home/jovyan/event_data/2018-11-03-events.csv', '/home/jovyan/event_data/2018-11-02-events.csv', '/home/jovyan/event_data/2018-11-26-events.csv', '/home/jovyan/event

In [6]:
rows_in_file = []
for files in file_path_list:
    df = pd.read_csv(files)

In [7]:
for values in df:
    print(values)

artist
auth
firstName
gender
itemInSession
lastName
length
level
location
method
page
registration
sessionId
song
status
ts
userId


In [42]:
for line in df:
    query = """INSERT INTO session_songs (sessionId, itemInSession, artist, song_title, song_length) VALUES(%s, %s, %s, %s, %s)"""
    artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
    session.execute(query, (int(sessionId), int(itemInSession), artist_name, song, float(length)))

ValueError: not enough values to unpack (expected 11, got 6)

In [52]:
for row in df.iterrows():
    

SyntaxError: incomplete input (755189656.py, line 2)

In [51]:
query = """INSERT INTO session_songs (sessionId, itemInSession, artist, song_title, song_length) VALUES(?, ?, ?, ?, ?)"""
prepared = session.prepare(query)

for item in df:
    session.execute(prepared,(item.itemInSession,item.artist,item.song,item.length))

AttributeError: 'str' object has no attribute 'itemInSession'

In [53]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [54]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO session_songs (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist_name, song, float(length)))

In [8]:
from queries import *

In [11]:
session.execute(CREATE_EVENT_LOG_TABLE)

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    print(next(csvreader))

['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']


In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        session.execute(INSERT_INTO_EVENT_LOG, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

In [16]:
rows = session.execute("""SELECT * FROM event_log""")

# for row in rows:
#     print(row.artist, row.song_title, row.song_length)

In [18]:
for row in rows:
    print(row.artist, row.song , row.first_name , row.last_name , row.user_id , row.session_id , row.item_in_session)

System of a Down Sad Statue Emily Benson 58 768 0
Ghostland Observatory Stranger Lover Emily Benson 58 768 1
Evergreen Terrace Zero Emily Benson 58 768 2
Deftones Head Up (LP Version) Kinsley Young 85 776 2
The Notorious B.I.G. Playa Hater (Amended Version) Kinsley Young 85 776 3
Orchestral Manoeuvres In The Dark Stay (The Black Rose And The Universal Wheel) Kinsley Young 85 776 4
Vangelis Roxane's Veil Kinsley Young 85 776 5
Spandau Ballet TRUE Kinsley Young 85 776 6
The Pussycat Dolls I'm Done Kinsley Young 85 776 8
The Hollies I'm Down Kinsley Young 85 776 9
Lily Allen LDN Kinsley Young 85 776 10
Fall Out Boy She's My Winona Kinsley Young 85 776 11
Metallica The Unforgiven II Kinsley Young 85 776 12
O-Zone Dragostea din tin (ma-ya-hi) Kinsley Young 85 776 13
Murs Bad Man! Kinsley Young 85 776 14
Basshunter Boten Anna [Radio edit] Kinsley Young 85 776 15
Maelo Ruiz Te Va A Doler Kinsley Young 85 776 16
Dwight Yoakam You're The One Kinsley Young 85 776 17
Fragma Everytime You Need Me 

In [20]:
session.execute(CREATE_SONG_USERS_TABLE)

In [21]:
file = 'event_datafile_new.csv'

df = pd.read_csv(file, usecols=[1, 4, 9, 10])
df.drop_duplicates(inplace=True)

for ix, row in df.iterrows():
    session.execute(INSERT_INTO_SONG_USERS, (row['song'], row['userId'], row['firstName'], row['lastName']))

In [22]:
rows = session.execute("""SELECT first_name, last_name FROM song_users WHERE song = 'All Hands Against His Own'""")

for row in rows:
    print( row.first_name, row.last_name )

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [44]:
session.shutdown()
cluster.shutdown()